In [1]:
import os
from google.colab import userdata

In [2]:
userdata = userdata.get('github')

os.environ['GITHUB_TOKEN'] = userdata

In [3]:
!git clone https://{userdata}@github.com/miguroi/sistech.git

Cloning into 'sistech'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 109 (delta 6), reused 13 (delta 4), pack-reused 90 (from 1)
Receiving objects: 100% (109/109), 50.12 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [4]:
cd sistech

/content/sistech


In [5]:
!ls data/csv

alt_coursera_courses_cleaned.csv  coursera_courses.csv
coursera_courses_cleaned.csv


In [6]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


### Setup

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

import random
from datetime import datetime, timedelta

In [8]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [9]:
original_df = pd.read_csv('data/csv/coursera_courses.csv')
original_df.head()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,language,course_id
0,Google Data Analytics,Google,4.763390,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Rmarkdown, Data Literacy, Data Visualization, Data Presentation, Data Ethics, Data Cleansing, Interactive Data Visualization, Data Validation, Ggplot2, Tableau Software, Sampling (Statistics), Presentations, Spreadsheet Software, Data Analysis, Data Visualization Software, Stakeholder Communications, LinkedIn, Interviewing Skills, Applicant Tracking Systems",/professional-certificates/google-data-analytics,False,NaN,s12n~kr43OcbTEeqeNBKhfgCLyw
1,Google Cybersecurity,Google,4.822850,53304,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Threat Modeling, Network Security, Incident Response, Vulnerability Management, Computer Security Incident Management, Hardening, Intrusion Detection and Prevention, Cyber Threat Intelligence, Threat Management, Cyber Attacks, Cybersecurity, Network Protocols, Cloud Security, Vulnerability Assessments, Bash (Scripting Language), Debugging, Linux, Interviewing Skills, Python Programming, SQL",/professional-certificates/google-cybersecurity,False,NaN,s12n~Dy6K-2UKEe2PIRJn6nL9pQ
2,Google Project Management:,Google,4.842615,130704,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Quality Management, Project Management Life Cycle, Requirements Analysis, Project Scoping, Project Closure, Project Management, Project Planning, Agile Project Management, Continuous Improvement Process, Project Controls, Backlogs, Stakeholder Communications, Milestones (Project Management), Quality Assessment, Team Management, Agile Methodology, Project Documentation, Change Management, Interviewing Skills, Applicant Tracking Systems",/professional-certificates/google-project-management,False,NaN,s12n~fq9UWMbTEeqpthJ2RmWGow
3,Google AI Essentials,Google,4.835467,2749,Beginner,Specialization,ONE_TO_THREE_MONTHS,"Prompt Engineering, Generative AI, Artificial Intelligence and Machine Learning (AI/ML), Large Language Modeling, Process Optimization, Productivity Software, Workforce Development, Digital Transformation, Innovation, Technical Writing, Emerging Technologies, Operational Efficiency, Business Solutions, Machine Learning Software, Data Security, Critical Thinking, Analysis, Data Analysis, Data Quality",/specializations/ai-essentials-google,False,NaN,s12n~3tzIujTqTk-YdcEFZ9r3sQ
4,Google Digital Marketing & E-commerce,Google,4.801080,40653,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Search Engine Marketing, Media Planning, Social Media Marketing, Google Ads, Email Marketing, Social Media Strategy, Search Engine Optimization, Order Fulfillment, Social Media Management, Performance Measurement, Spreadsheet Software, A/B Testing, Customer Retention, E-Commerce, Campaign Management, Loyalty Programs, Marketing, Interviewing Skills, Applicant Tracking Systems",/professional-certificates/google-digital-marketing-ecommerce,False,NaN,s12n~aYYrIEl-EeyCjQ5Y8Mzdsw


In [10]:
clean_df = pd.read_csv('data/csv/coursera_courses_cleaned.csv')

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1704 non-null   object 
 1   organization  1704 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1704 non-null   int64  
 4   difficulty    1704 non-null   object 
 5   course_type   1704 non-null   object 
 6   duration      1704 non-null   object 
 7   skills        1704 non-null   object 
 8   url           1704 non-null   object 
 9   is_free       1704 non-null   bool   
 10  course_id     1704 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 134.9+ KB


In [11]:
clean_df = clean_df.rename(columns={'rating': 'course_rating'})
clean_df = clean_df.rename(columns={'title': 'clean_title'})

In [12]:
clean_df = pd.merge(clean_df, original_df[['course_id', 'title']], on='course_id', how='left')

clean_df.head()

,clean_title,organization,course_rating,review_count,difficulty,course_type,duration,skills,url,is_free,course_id,title
0,google data analytics,Google,4.763390,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems,/professional-certificates/google-data-analytics,False,s12n~kr43OcbTEeqeNBKhfgCLyw,Google Data Analytics
1,google cybersecurity,Google,4.822850,53304,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocol cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql,/professional-certificates/google-cybersecurity,False,s12n~Dy6K-2UKEe2PIRJn6nL9pQ,Google Cybersecurity
2,google project_management,Google,4.842615,130704,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,quality management_project_management life_cycle requirements_analysis project scoping project_closure project_management_project planning agile_project_management continuous_improvement_process project control backlog stakeholder_communications milestones_project_management quality assessment team_management agile_methodology_project documentation change_management interviewing_skills applicant_tracking_systems,/professional-certificates/google-project-management,False,s12n~fq9UWMbTEeqpthJ2RmWGow,Google Project Management:
3,google ai essential,Google,4.835467,2749,Beginner,Specialization,ONE_TO_THREE_MONTHS,prompt_engineering_generative ai artificial_intelligence machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solution machine learning_software data_security critical_thinking analysis_data_analysis data_quality,/specializations/ai-essentials-google,False,s12n~3tzIujTqTk-YdcEFZ9r3sQ,Google AI Essentials
4,google digital_marketing ecommerce,Google,4.801080,40653,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,data storytelling search_engine_marketing medium planning social_media_marketing google ad email_marketing social_media_strategy search_engine_optimization order fulfillment social_media management_performance_measurement spreadsheet_software testing customer_retention ecommerce campaign_management loyalty program marketing interviewing_skills applicant_tracking_systems,/professional-certificates/google-digital-marketing-ecommerce,False,s12n~aYYrIEl-EeyCjQ5Y8Mzdsw,Google Digital Marketing & E-commerce


### Create Synthetic User Dataset

In [13]:
course_ids = clean_df['course_id'].tolist()

num_users = 500
user_ids = [f"user_{i}" for i in range(1, num_users + 1)]

In [14]:
data = []
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

In [15]:
for user_id in user_ids:
    num_interactions = random.randint(5, 50)
    for _ in range(num_interactions):
        course_id = random.choice(course_ids)

        is_click = random.choices([True, False], weights=[0.7, 0.3], k=1)[0]

        is_enroll = False
        if is_click:
            is_enroll = random.choices([True, False], weights=[0.5, 0.5], k=1)[0]

        rating = np.nan
        if is_enroll:
            rating = random.randint(1, 5)

        random_days = random.randint(0, (end_date - start_date).days)
        interaction_date = start_date + timedelta(days=random_days)

        data.append({
            "user_id": user_id,
            "course_id": course_id,
            "is_click": is_click,
            "is_enroll": is_enroll,
            "user_rating": rating,
            "interaction_date": interaction_date
        })

In [16]:
synthetic_user_df = pd.DataFrame(data)

In [17]:
synthetic_user_df = pd.merge(synthetic_user_df, clean_df[['course_id']], on='course_id', how='left')

synthetic_user_df.head()

,user_id,course_id,is_click,is_enroll,user_rating,interaction_date
0,user_1,course~UU2MxlcmEee91g5WFbHAAA,True,False,NaN,2024-12-20
1,user_1,course~lYw3nxuGEeWo5g5SXpDA8Q,True,False,NaN,2024-08-19
2,user_1,course~zMg9YpBtEeiWbhI9enrBdA,True,False,NaN,2024-05-27
3,user_1,course~9voo3M_1Ee2ungry-zmhYQ,True,False,NaN,2024-08-13
4,user_1,course~XANaz8dEEey3GgqQBm7DaQ,True,True,1.0,2024-02-24


In [18]:
synthetic_user_df.to_csv('synthetic_user_dataset.csv', index=False)

### Text Vectorization

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
from sklearn.preprocessing import MinMaxScaler

In [20]:
# title
tfidf_title = TfidfVectorizer(stop_words='english')
tfidf_matrix_title = tfidf_title.fit_transform(clean_df['clean_title'])

# skills
tfidf_skills = TfidfVectorizer(stop_words='english')
tfidf_matrix_skills = tfidf_skills.fit_transform(clean_df['skills'])

### Features Normalization

In [21]:
scaler = MinMaxScaler()

numerical_features = ['course_rating', 'review_count']
numerical_normalized = scaler.fit_transform(clean_df[numerical_features])

In [22]:
combined_features = hstack([
    tfidf_matrix_title,
    tfidf_matrix_skills,
    numerical_normalized
])

In [23]:
combined_features.shape

(1704, 4477)

In [24]:
combined_features_df = pd.DataFrame(combined_features.todense())

combined_features_df.to_csv('vectorization_result.csv', index=False)

combined_features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4467,4468,4469,4470,4471,4472,4473,4474,4475,4476
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.940848,0.613024
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.955713,0.191884
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.960654,0.470515
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.958867,0.009892
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.950270,0.146342


### Calculate Similarity

In [25]:
cosine_sim = cosine_similarity(combined_features)

In [26]:
cosine_sim.shape

(1704, 1704)

In [27]:
similarity_df = pd.DataFrame(cosine_sim, index=clean_df['course_id'], columns=clean_df['title'])
similarity_df.head()

title,Google Data Analytics,Google Cybersecurity,Google Project Management:,Google AI Essentials,Google Digital Marketing & E-commerce,Google IT Support,Google UX Design,Machine Learning,Google Prompting Essentials,AI For Everyone,...,Health Care Innovation,Drug Safety: Pharmacovigilance,Version Control with Git,Supply Chain Management Strategy,Environmental Management & Ethics,America's Unwritten Constitution,Introduction to Ableton Live,Fixing Healthcare Delivery 2.0: Advanced Lean,Simulation and modeling of natural processes,Operations and Patient Safety for Healthcare IT Staff
course_id,,,,,,,,,,,,,,,,,,,,,
s12n~kr43OcbTEeqeNBKhfgCLyw,1.000000,0.453059,0.520727,0.397387,0.437263,0.523191,0.475553,0.319158,0.380996,0.323018,...,0.278071,0.296919,0.300566,0.302949,0.283197,0.291403,0.290022,0.286689,0.261341,0.292996
s12n~Dy6K-2UKEe2PIRJn6nL9pQ,0.453059,1.000000,0.473094,0.437852,0.423057,0.470643,0.443132,0.322750,0.406973,0.319467,...,0.296807,0.317092,0.307825,0.310705,0.302224,0.311062,0.309142,0.306151,0.294914,0.320647
s12n~fq9UWMbTEeqpthJ2RmWGow,0.520727,0.473094,1.000000,0.425221,0.444819,0.515728,0.473754,0.325547,0.395048,0.326973,...,0.289091,0.308745,0.300705,0.310541,0.294402,0.302960,0.301368,0.298102,0.271675,0.304632
s12n~3tzIujTqTk-YdcEFZ9r3sQ,0.397387,0.437852,0.425221,1.000000,0.397289,0.392297,0.405364,0.335760,0.554212,0.491301,...,0.310056,0.319804,0.309786,0.333069,0.304717,0.313662,0.311536,0.308761,0.310468,0.315442
s12n~aYYrIEl-EeyCjQ5Y8Mzdsw,0.437263,0.423057,0.444819,0.397289,1.000000,0.421317,0.419287,0.320448,0.375549,0.316402,...,0.296398,0.316673,0.307254,0.318329,0.301802,0.310635,0.308672,0.305744,0.278491,0.312383


### Recommendation System Modeling

In [28]:
synthetic_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13554 entries, 0 to 13553
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           13554 non-null  object        
 1   course_id         13554 non-null  object        
 2   is_click          13554 non-null  bool          
 3   is_enroll         13554 non-null  bool          
 4   user_rating       4766 non-null   float64       
 5   interaction_date  13554 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), float64(1), object(2)
memory usage: 450.2+ KB


In [29]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clean_title    1704 non-null   object 
 1   organization   1704 non-null   object 
 2   course_rating  1704 non-null   float64
 3   review_count   1704 non-null   int64  
 4   difficulty     1704 non-null   object 
 5   course_type    1704 non-null   object 
 6   duration       1704 non-null   object 
 7   skills         1704 non-null   object 
 8   url            1704 non-null   object 
 9   is_free        1704 non-null   bool   
 10  course_id      1704 non-null   object 
 11  title          1704 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 148.2+ KB


In [30]:
sample_course_id = clean_df['course_id'].iloc[0]
sample_course_name = clean_df['title'].iloc[0]

print(f"Sample Course ID: {sample_course_id}")
print(f"Sample Course Name: {sample_course_name}")

Sample Course ID: s12n~kr43OcbTEeqeNBKhfgCLyw
Sample Course Name: Google Data Analytics


In [31]:
sample_user_id = synthetic_user_df['user_id'].iloc[0]

print(f"Sample User ID: {sample_user_id}")

print(f"Sample User Preferences for {sample_user_id}: ")
synthetic_user_df[synthetic_user_df['user_id'] == sample_user_id]

Sample User ID: user_1
Sample User Preferences for user_1: 


,user_id,course_id,is_click,is_enroll,user_rating,interaction_date
0,user_1,course~UU2MxlcmEee91g5WFbHAAA,True,False,NaN,2024-12-20
1,user_1,course~lYw3nxuGEeWo5g5SXpDA8Q,True,False,NaN,2024-08-19
2,user_1,course~zMg9YpBtEeiWbhI9enrBdA,True,False,NaN,2024-05-27
3,user_1,course~9voo3M_1Ee2ungry-zmhYQ,True,False,NaN,2024-08-13
4,user_1,course~XANaz8dEEey3GgqQBm7DaQ,True,True,1.0,2024-02-24
5,user_1,s12n~XTZ9ijPWRW65T4LrV2aFfw,True,True,2.0,2024-12-31
6,user_1,s12n~Tpnq4Qv-RdO7rVtMrX2_DA,True,False,NaN,2024-07-05
7,user_1,course~qeyXSV_uEeiNIQqyU-T6bA,True,False,NaN,2024-04-20
8,user_1,s12n~P9ChEotMRLO280OmpUbLEw,True,False,NaN,2024-11-25
9,user_1,s12n~oro2HY5DRfGO_t1HZ2Olsw,False,False,NaN,2024-10-28


#### Content-based Filtering Recommendation System

In [32]:
course_id_to_index = {course_id: idx for idx, course_id in enumerate(clean_df['course_id'])}
course_index = course_id_to_index[sample_course_id]

list(course_id_to_index.items())[:10]

[('s12n~kr43OcbTEeqeNBKhfgCLyw', 0),
 ('s12n~Dy6K-2UKEe2PIRJn6nL9pQ', 1),
 ('s12n~fq9UWMbTEeqpthJ2RmWGow', 2),
 ('s12n~3tzIujTqTk-YdcEFZ9r3sQ', 3),
 ('s12n~aYYrIEl-EeyCjQ5Y8Mzdsw', 4),
 ('s12n~7lHCSlFIEeeffRIHljDI_g', 5),
 ('s12n~Z-5wCcbTEeqeNBKhfgCLyw', 6),
 ('s12n~nOOfCDWeEeuiZgo2K4rorQ', 7),
 ('s12n~1a_Qa92eQkOSjt6VLZ6lsg', 8),
 ('course~daG-a-O1EeijKBISCWxf6g', 9)]

In [33]:
course_id_to_index = {course_id: idx for idx, course_id in enumerate(clean_df['course_id'])}
course_index = course_id_to_index[sample_course_id]

similarity_scores = cosine_sim[course_index]

course_similarities = pd.Series(similarity_scores, index=clean_df['course_id'])
course_similarities.name = 'similarity'
course_similarities = pd.DataFrame(course_similarities)

content_recommendations = pd.merge(course_similarities, clean_df[['course_id', 'title']], left_index=True, right_on='course_id')
content_recommendations.head()

content_recommendations.set_index('course_id', inplace=True)
content_recommendations.head()

,similarity,title
course_id,,
s12n~kr43OcbTEeqeNBKhfgCLyw,1.000000,Google Data Analytics
s12n~Dy6K-2UKEe2PIRJn6nL9pQ,0.453059,Google Cybersecurity
s12n~fq9UWMbTEeqpthJ2RmWGow,0.520727,Google Project Management:
s12n~3tzIujTqTk-YdcEFZ9r3sQ,0.397387,Google AI Essentials
s12n~aYYrIEl-EeyCjQ5Y8Mzdsw,0.437263,Google Digital Marketing & E-commerce


In [34]:
content_recommendations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, s12n~kr43OcbTEeqeNBKhfgCLyw to course~z7bOth-eEeuQ9xLSu4r6yQ
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   similarity  1704 non-null   float64
 1   title       1704 non-null   object 
dtypes: float64(1), object(1)
memory usage: 39.9+ KB


In [35]:
print(f"Top 10 Recommended Courses for {sample_course_name} using Content-based Filtering:")
content_recommendations.head(10)

Top 10 Recommended Courses for Google Data Analytics using Content-based Filtering:


,similarity,title
course_id,,
s12n~kr43OcbTEeqeNBKhfgCLyw,1.000000,Google Data Analytics
s12n~Dy6K-2UKEe2PIRJn6nL9pQ,0.453059,Google Cybersecurity
s12n~fq9UWMbTEeqpthJ2RmWGow,0.520727,Google Project Management:
s12n~3tzIujTqTk-YdcEFZ9r3sQ,0.397387,Google AI Essentials
s12n~aYYrIEl-EeyCjQ5Y8Mzdsw,0.437263,Google Digital Marketing & E-commerce
s12n~7lHCSlFIEeeffRIHljDI_g,0.523191,Google IT Support
s12n~Z-5wCcbTEeqeNBKhfgCLyw,0.475553,Google UX Design
s12n~nOOfCDWeEeuiZgo2K4rorQ,0.319158,Machine Learning
s12n~1a_Qa92eQkOSjt6VLZ6lsg,0.380996,Google Prompting Essentials


In [36]:
content_recommendations_csv = content_recommendations.to_csv('content_recommendations.csv')
print(f"Content Recommendations saved to content_recommendations.csv")

Content Recommendations saved to content_recommendations.csv


#### Collaborative Filtering Recommendation System

In [37]:
course_user_merged = pd.merge(synthetic_user_df, clean_df, on='course_id', how='left')
course_user_merged.head()

,user_id,course_id,is_click,is_enroll,user_rating,interaction_date,clean_title,organization,course_rating,review_count,difficulty,course_type,duration,skills,url,is_free,title
0,user_1,course~UU2MxlcmEee91g5WFbHAAA,True,False,NaN,2024-12-20,medium ethic governance,University of Amsterdam,4.771529,569,Intermediate,Course,ONE_TO_FOUR_WEEKS,ethical_standards conduct medium communications_business ethic research governance journalism data_ethics advertising,/learn/media-ethics-governance,True,Media ethics & governance
1,user_1,course~lYw3nxuGEeWo5g5SXpDA8Q,True,False,NaN,2024-08-19,cryptography,Stanford University,4.801081,4439,Mixed,Course,ONE_TO_THREE_MONTHS,cryptography data encryption standard encryption key_management advanced encryption standard aes public key cryptography_standards_pkcs cybersecurity authentication data_integrity secure coding algorithm,/learn/crypto,True,Cryptography I
2,user_1,course~zMg9YpBtEeiWbhI9enrBdA,True,False,NaN,2024-05-27,site reliability_engineering measuring managing reliability,Google Cloud,4.547290,941,Intermediate,Course,ONE_TO_THREE_MONTHS,site reliability_engineering service_level devops system monitoring continuous_monitoring risk_management_framework cloud_computing software_documentation,/learn/site-reliability-engineering-slos,False,Site Reliability Engineering: Measuring and Managing Reliability
3,user_1,course~9voo3M_1Ee2ungry-zmhYQ,True,False,NaN,2024-08-13,web blockchain fundamental,INSEAD,4.656566,99,Mixed,Course,ONE_TO_FOUR_WEEKS,digital_assets blockchain emerging_technologies fintech business_transformation digital_transformation ecommerce technical_design scalability business_strategy governance,/learn/web3-blockchain-fundamentals,False,Web3 and Blockchain Fundamentals
4,user_1,course~XANaz8dEEey3GgqQBm7DaQ,True,True,1.0,2024-02-24,coding interview preparation,Meta,4.647226,703,Intermediate,Course,ONE_TO_FOUR_WEEKS,data_structure algorithm software visualization pseudocode programming_principles computational_thinking computer_science technical_communication computational_logic program_development communication,/learn/coding-interview-preparation,False,Coding Interview Preparation


In [38]:
ratings_matrix = synthetic_user_df.pivot_table(index=['course_id'],columns=['user_id'],values='user_rating', fill_value=0)
ratings_matrix.head(3)

user_id,user_1,user_10,user_100,user_101,user_102,user_103,user_104,user_105,user_106,user_107,...,user_90,user_91,user_92,user_93,user_94,user_95,user_96,user_97,user_98,user_99
course_id,,,,,,,,,,,,,,,,,,,,,
course~-1YwAnTLEeSjmyIAC0aXFg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
course~-28Ns4tvEeiqpQ4udDR_iA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
course~-3PuGcl0Eeuiewq9VRQV-Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [39]:
course_similarity = cosine_similarity(ratings_matrix.values)
np.fill_diagonal(course_similarity, 0)
course_similarity

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
course_similarity_df = pd.DataFrame(course_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

course_similarity_df.head()

course_id,course~-1YwAnTLEeSjmyIAC0aXFg,course~-28Ns4tvEeiqpQ4udDR_iA,course~-3PuGcl0Eeuiewq9VRQV-Q,course~-6EEg0s4Ee-4ug4_QNLLSQ,course~-7ZymCvBEee6gA5XksfBbg,course~-8cOpydnEeaTvRKF2VtmxQ,course~-A_ToPNPEeSAEiIAC9TCSQ,course~-CoDwy4CEe-GuAr_9hcaqw,course~-Eu38u08EeSKeyIACwQXPg,course~-FCmAR6kEfCpIQr_6QDb7Q,...,s12n~zNtJTbPUEeWVOwrJiBe1eQ,s12n~zQ29Z0cJEey1jhLpt2nhVQ,s12n~zRnwN2HTSqeM6PwLXqKLRg,s12n~zb8J0mWKEeaomwoYfGdF7Q,s12n~zgGHK0SjRIaYm5j9ptmjfA,s12n~zh7400XtEeWUww73KBYvPw,s12n~zj2VppjQEeWh0Q5bBaG7rw,s12n~zjWaXRMhEeudwgrn4wF8EQ,s12n~zkV7-RZKEea3nArFC-QKGw,s12n~zn_b0Y_lEeqE2Q5NANyNFw
course_id,,,,,,,,,,,,,,,,,,,,,
course~-1YwAnTLEeSjmyIAC0aXFg,0.0,0.0,0.0,0.0,0.0,0.089572,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.499756,0.0,0.0,0.00000,0.0,0.0,0.0
course~-28Ns4tvEeiqpQ4udDR_iA,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.23281,0.0,0.0,0.0
course~-3PuGcl0Eeuiewq9VRQV-Q,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0
course~-6EEg0s4Ee-4ug4_QNLLSQ,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0
course~-7ZymCvBEee6gA5XksfBbg,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0


In [41]:
collaborative_recommendations = course_similarity_df.loc[sample_course_id].sort_values(ascending=False)

collaborative_recommendations.name = 'similarity'
collaborative_recommendations = pd.DataFrame(collaborative_recommendations)

collaborative_recommendations = pd.merge(collaborative_recommendations, clean_df[['course_id', 'title']], left_index=True, right_on='course_id')
collaborative_recommendations = collaborative_recommendations.set_index('course_id')
collaborative_recommendations = collaborative_recommendations[['title', 'similarity']]

print(f"Top 10 Recommended Courses for {sample_course_name} using Collaborative Filtering:")
collaborative_recommendations.head(10)

Top 10 Recommended Courses for Google Data Analytics using Collaborative Filtering:


,title,similarity
course_id,,
s12n~mMVai3AYQJ6HgSjJlbPJ7A,Mechanization in Construction,0.970143
s12n~mZJh7rLcEe2DNQpOhYGSRQ,Human Anatomy & Physiology I,0.846810
s12n~87ESinHoQRWhM0F8mKwKBA,Strategies for Heavy Lifting,0.685994
course~p79Awdv8EeeCEw5jIEhNXg,3D CAD Application,0.631509
course~3obxa2UJEe2NnA4jep2fLw,Sound the Alarm: Detection and Response,0.579771
course~PFbtELlwEe-eKxJKZPOGuw,Change Management,0.462497
s12n~-s180RslQ8SG59v5P2kV2g,IBM and ISC2 Cybersecurity Specialist,0.306786
s12n~y115AxFeQG27plEdIAZzPQ,Human Anatomy and Medical Terminology Fundamentals,0.242536
course~40P-6j0PEeuwehK5Ir7QEw,Entrepreneurial Mindset,0.242536


In [42]:
collaborative_recommendations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1604 entries, s12n~mMVai3AYQJ6HgSjJlbPJ7A to course~ZOaxMgNuEe6avhLjR2t3Lw
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1604 non-null   object 
 1   similarity  1604 non-null   float64
dtypes: float64(1), object(1)
memory usage: 37.6+ KB


In [43]:
collaborative_recommendations.head()

,title,similarity
course_id,,
s12n~mMVai3AYQJ6HgSjJlbPJ7A,Mechanization in Construction,0.970143
s12n~mZJh7rLcEe2DNQpOhYGSRQ,Human Anatomy & Physiology I,0.846810
s12n~87ESinHoQRWhM0F8mKwKBA,Strategies for Heavy Lifting,0.685994
course~p79Awdv8EeeCEw5jIEhNXg,3D CAD Application,0.631509
course~3obxa2UJEe2NnA4jep2fLw,Sound the Alarm: Detection and Response,0.579771


In [44]:
collaborative_recommendations_csv = collaborative_recommendations.to_csv('collaborative_recommendations.csv')
print(f"Collaborative Recommendations saved to collaborative_recommendations.csv")

Collaborative Recommendations saved to collaborative_recommendations.csv


#### Hybrid Systems Recommendation System

In [45]:
CONTENT_WEIGHT = 0.6
COLLABORATIVE_WEIGHT = 0.4

In [46]:
course_user_merged = pd.merge(synthetic_user_df, clean_df, on='course_id', how='left')

ratings_matrix = synthetic_user_df.pivot_table(index=['course_id'],columns=['user_id'],values='user_rating', fill_value=0)

hybrid_recommendations = cosine_similarity(ratings_matrix.values)

np.fill_diagonal(hybrid_recommendations, 0)

hybrid_recommendations_df = pd.DataFrame(hybrid_recommendations, index=ratings_matrix.index, columns=ratings_matrix.index)

In [47]:
def get_recommendations(course_id, content_weight=None, collaborative_weight=None, n=10):
    if course_id not in content_recommendations.index:
        print(f"Course {course_id} not found in content recommendations")
        return pd.DataFrame()

    if course_id not in collaborative_recommendations.index:
        print(f"Course {course_id} not found in collaborative recommendations")
        return pd.DataFrame()

    content_scores = content_recommendations['similarity']
    collab_scores = collaborative_recommendations['similarity']

    scores_df = pd.DataFrame({
        'content': content_scores,
        'collaborative': collab_scores
    }).fillna(0)

    scores_df['hybrid_score'] = (content_weight * scores_df['content'] +
                                collaborative_weight * scores_df['collaborative'])

    recommendations = scores_df['hybrid_score'].sort_values(ascending=False)
    recommendations = recommendations.drop(course_id, errors='ignore').head(n)

    result = pd.DataFrame(recommendations)
    result = result.merge(clean_df[['course_id', 'title']],
                         left_index=True, right_on='course_id')
    result = result.set_index('course_id')[['title', 'hybrid_score']]

    return result

hybrid_recommendations = get_recommendations(sample_course_id, CONTENT_WEIGHT, COLLABORATIVE_WEIGHT)

print(f"Top 10 Recommended Courses for {sample_course_name} using Hybrid Filtering:")

hybrid_recommendations.head(10)

Top 10 Recommended Courses for Google Data Analytics using Hybrid Filtering:


,title,hybrid_score
course_id,,
s12n~mMVai3AYQJ6HgSjJlbPJ7A,Mechanization in Construction,0.552535
s12n~mZJh7rLcEe2DNQpOhYGSRQ,Human Anatomy & Physiology I,0.512747
s12n~87ESinHoQRWhM0F8mKwKBA,Strategies for Heavy Lifting,0.436979
course~p79Awdv8EeeCEw5jIEhNXg,3D CAD Application,0.433651
course~3obxa2UJEe2NnA4jep2fLw,Sound the Alarm: Detection and Response,0.406850
s12n~kx6DXkl-EeyQ6Qp2PlG9FQ,Google Advanced Data Analytics,0.384994
s12n~agEcxHgeRASq8yHxWPRSWA,Google Cloud Data Analytics,0.358302
course~PFbtELlwEe-eKxJKZPOGuw,Change Management,0.352451
course~ZWYBq9zVEeqMbw6VbjEnNw,Introduction to Data Analytics,0.337605


In [48]:
hybrid_recommendations_csv = hybrid_recommendations.to_csv('hybrid_recommendations.csv')
print(f"Hybrid Recommendations saved to hybrid_recommendations.csv")

Hybrid Recommendations saved to hybrid_recommendations.csv


### Result

In [49]:
synthetic_user_df.head()

,user_id,course_id,is_click,is_enroll,user_rating,interaction_date
0,user_1,course~UU2MxlcmEee91g5WFbHAAA,True,False,NaN,2024-12-20
1,user_1,course~lYw3nxuGEeWo5g5SXpDA8Q,True,False,NaN,2024-08-19
2,user_1,course~zMg9YpBtEeiWbhI9enrBdA,True,False,NaN,2024-05-27
3,user_1,course~9voo3M_1Ee2ungry-zmhYQ,True,False,NaN,2024-08-13
4,user_1,course~XANaz8dEEey3GgqQBm7DaQ,True,True,1.0,2024-02-24


In [50]:
content_recommendations.head()

,similarity,title
course_id,,
s12n~kr43OcbTEeqeNBKhfgCLyw,1.000000,Google Data Analytics
s12n~Dy6K-2UKEe2PIRJn6nL9pQ,0.453059,Google Cybersecurity
s12n~fq9UWMbTEeqpthJ2RmWGow,0.520727,Google Project Management:
s12n~3tzIujTqTk-YdcEFZ9r3sQ,0.397387,Google AI Essentials
s12n~aYYrIEl-EeyCjQ5Y8Mzdsw,0.437263,Google Digital Marketing & E-commerce


In [51]:
collaborative_recommendations.head()

,title,similarity
course_id,,
s12n~mMVai3AYQJ6HgSjJlbPJ7A,Mechanization in Construction,0.970143
s12n~mZJh7rLcEe2DNQpOhYGSRQ,Human Anatomy & Physiology I,0.846810
s12n~87ESinHoQRWhM0F8mKwKBA,Strategies for Heavy Lifting,0.685994
course~p79Awdv8EeeCEw5jIEhNXg,3D CAD Application,0.631509
course~3obxa2UJEe2NnA4jep2fLw,Sound the Alarm: Detection and Response,0.579771


In [52]:
hybrid_recommendations.head()

,title,hybrid_score
course_id,,
s12n~mMVai3AYQJ6HgSjJlbPJ7A,Mechanization in Construction,0.552535
s12n~mZJh7rLcEe2DNQpOhYGSRQ,Human Anatomy & Physiology I,0.512747
s12n~87ESinHoQRWhM0F8mKwKBA,Strategies for Heavy Lifting,0.436979
course~p79Awdv8EeeCEw5jIEhNXg,3D CAD Application,0.433651
course~3obxa2UJEe2NnA4jep2fLw,Sound the Alarm: Detection and Response,0.406850
